<a href="https://colab.research.google.com/github/Nikhilsai-123/dailyassesment/blob/main/pspyk3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark Practice Notebook
Experimenting further with Spark Dataframes

In [ ]:
#Create CSV files for the following Datasets

patients
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

visits
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

In [ ]:
%%writefile patients.csv
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

Writing patients.csv


In [ ]:
%%writefile visits.csv
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

Writing visits.csv


In [44]:
#Import necessary spark libraries
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [12]:
%%writefile patients.csv
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

Writing patients.csv


In [13]:
%%writefile visits.csv
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

Writing visits.csv


In [15]:
#Create a SparkSession
spark=SparkSession.builder.appName("SparkPractice").getOrCreate()
df=spark.read.option('header',True).csv('/content/patients.csv')
df1=spark.read.option('header',True).csv('/content/visits.csv')

In [16]:

#Create dataframes from CSV files using enforced Schema
patientSchema=StructType([
    StructField('patientId',IntegerType(),True),
    StructField('firstName',StringType(),True),
    StructField('lastName',StringType(),True),
    StructField('age',IntegerType(),True)
])
visitSchema=StructType([
    StructField('visitId',IntegerType(),True),
    StructField('patientId',IntegerType(),True),
    StructField('visitDuration',IntegerType(),True)
])
#Schems Directive [Names as string, everything else as integers]
df.show()
df1.show()

+---------+---------+--------+---+
|patientId|firstName|lastName|age|
+---------+---------+--------+---+
|      101|    Alice|   Smith| 30|
|      102|      Bob| Johnson| 45|
|      103|  Charlie|Williams| 50|
|      104|     John|   Smith| 78|
+---------+---------+--------+---+

+-------+---------+-------------+
|visitId|patientId|visitDuration|
+-------+---------+-------------+
|      1|      101|           15|
|      2|      101|           30|
|      3|      102|           45|
|      4|      102|           30|
|      5|      104|           20|
|      6|      103|           60|
|      7|      103|           50|
|      8|      104|           45|
|      9|     NULL|           45|
|     10|      101|           30|
|     11|      103|           40|
|     12|     NULL|           30|
|     13|      104|           25|
|     15|      102|           15|
+-------+---------+-------------+



In [22]:
#Display all the valid visits
df1.filter(df1['patientId']!='NULL').show()

#Fetch the total number of valid visits
df1.filter(df1['patientId']!='NULL').count()

+-------+---------+-------------+
|visitId|patientId|visitDuration|
+-------+---------+-------------+
|      1|      101|           15|
|      2|      101|           30|
|      3|      102|           45|
|      4|      102|           30|
|      5|      104|           20|
|      6|      103|           60|
|      7|      103|           50|
|      8|      104|           45|
|     10|      101|           30|
|     11|      103|           40|
|     13|      104|           25|
|     15|      102|           15|
+-------+---------+-------------+



12

In [40]:
#Get the total hours of patient visit
x=df1.filter(df1['patientId']!='NULL')
x.groupBy('patientId').agg({'visitDuration':'sum'}).show()

+---------+------------------+
|patientId|sum(visitDuration)|
+---------+------------------+
|      101|              75.0|
|      104|              90.0|
|      102|              90.0|
|      103|             150.0|
+---------+------------------+



In [42]:
#Fetch the top two most visited patients on the basis of occurances
pv=df.join(df1,'patientId')
pv.groupBy('patientId','firstName').count().orderBy('count',ascending=False).show()

+---------+---------+-----+
|patientId|firstName|count|
+---------+---------+-----+
|      102|      Bob|    3|
|      103|  Charlie|    3|
|      101|    Alice|    3|
|      104|     John|    3|
+---------+---------+-----+



In [46]:
#Add a new column patientFullName & display complete dataframe
pv.withColumn('patientFullName',concat(col('firstName'),col('lastName'))).show()

+---------+---------+--------+---+-------+-------------+---------------+
|patientId|firstName|lastName|age|visitId|visitDuration|patientFullName|
+---------+---------+--------+---+-------+-------------+---------------+
|      101|    Alice|   Smith| 30|      1|           15|     AliceSmith|
|      101|    Alice|   Smith| 30|      2|           30|     AliceSmith|
|      102|      Bob| Johnson| 45|      3|           45|     BobJohnson|
|      102|      Bob| Johnson| 45|      4|           30|     BobJohnson|
|      104|     John|   Smith| 78|      5|           20|      JohnSmith|
|      103|  Charlie|Williams| 50|      6|           60|CharlieWilliams|
|      103|  Charlie|Williams| 50|      7|           50|CharlieWilliams|
|      104|     John|   Smith| 78|      8|           45|      JohnSmith|
|      101|    Alice|   Smith| 30|     10|           30|     AliceSmith|
|      103|  Charlie|Williams| 50|     11|           40|CharlieWilliams|
|      104|     John|   Smith| 78|     13|         

In [53]:
#Display the name of patients with maximum visiting hours
maxvisit=pv.groupBy('patientId','firstName','lastName','age').agg({'visitDuration':'sum'})
maxvisit.orderBy('sum(visitDuration)',ascending=False).show(1)


+---------+---------+--------+---+------------------+
|patientId|firstName|lastName|age|sum(visitDuration)|
+---------+---------+--------+---+------------------+
|      103|  Charlie|Williams| 50|             150.0|
+---------+---------+--------+---+------------------+
only showing top 1 row



In [54]:
#Find the patient with maximum visiting hours in a single visit
pv.orderBy('visitDuration',ascending=False).show(1)

+---------+---------+--------+---+-------+-------------+
|patientId|firstName|lastName|age|visitId|visitDuration|
+---------+---------+--------+---+-------+-------------+
|      103|  Charlie|Williams| 50|      6|           60|
+---------+---------+--------+---+-------+-------------+
only showing top 1 row



In [55]:
#Show the patient info according to age-seniority
pv.orderBy('age',ascending=False).show()

+---------+---------+--------+---+-------+-------------+
|patientId|firstName|lastName|age|visitId|visitDuration|
+---------+---------+--------+---+-------+-------------+
|      104|     John|   Smith| 78|      5|           20|
|      104|     John|   Smith| 78|      8|           45|
|      104|     John|   Smith| 78|     13|           25|
|      103|  Charlie|Williams| 50|      6|           60|
|      103|  Charlie|Williams| 50|      7|           50|
|      103|  Charlie|Williams| 50|     11|           40|
|      102|      Bob| Johnson| 45|      3|           45|
|      102|      Bob| Johnson| 45|      4|           30|
|      102|      Bob| Johnson| 45|     15|           15|
|      101|    Alice|   Smith| 30|      1|           15|
|      101|    Alice|   Smith| 30|      2|           30|
|      101|    Alice|   Smith| 30|     10|           30|
+---------+---------+--------+---+-------+-------------+

